In [15]:
# pdp package path
import os, sys
sys.path.insert(1, os.path.join(os.getcwd()  , '../..'))

# imports
import random
from collections import defaultdict
from core import * 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [16]:
## Define problems with sizes 10, 15, 20
# Each with an id 1, 2, 3, ... where (id // 3) * 5 + 10 = number of requests

possible_items = [
    TwoDimensionalItem(300, 400),
    TwoDimensionalItem(600, 300),
    TwoDimensionalItem(600, 400),
    TwoDimensionalItem(300, 200),
]

# 2 small compartments and 2 large compartments
small_vehicle = Vehicle(
    compartments=[
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 600),
        TwoDimensionalCompartment(800, 600), 
    ]
)

# 3 small compartments and 2 large compartments
medium_vehicle = Vehicle(
    compartments=[
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 600),
        TwoDimensionalCompartment(800, 600), 
    ]
)

# 3 small compartments and 3 large compartments
large_vehicle = Vehicle(
    compartments=[
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 600),
        TwoDimensionalCompartment(800, 600),
        TwoDimensionalCompartment(800, 600),  
    ]
)

vehicles = [small_vehicle, medium_vehicle, large_vehicle]

group_size = 3
instance_sizes = [10, 15, 20]
problem_ids = list(range(group_size * len(instance_sizes)))

# create random problems with seed based on id!
problems = []
modelled_problems = []
for prob_id in problem_ids:
    instance_size_idx = prob_id // group_size
    instance_size = instance_sizes[instance_size_idx]
    nb_items = instance_size
    vehicle = vehicles[instance_size_idx]
    V = list(range(0, nb_items * 2 + 1))
    random.seed(prob_id)
    items = random.choices(possible_items, k=nb_items)
    random.seed(prob_id)
    distance_matrix = [
        [random.randint(1, 100) if i != 0 and j!=0 and i != j else 0 for i in V]
        for j in V
    ]
    
    problem = TwoDimensionalProblem(items, vehicle, distance_matrix, name=f"prob_{prob_id}")
    problems.append(problem)

    # Modellel Problem for time-limited gurobi
    modelled_problem = ModelledTwoDimensionalProblem(items, vehicle, distance_matrix=distance_matrix)
    modelled_problem.create_model()
    modelled_problem.apply_constraints()
    modelled_problem.set_model_objective()
    modelled_problems.append(modelled_problem)


In [17]:
# Initial, quick solutions by gurobi
initial_solutions_dict = {}
for prob_id, problem in enumerate(modelled_problems):
    initial_solution_time_limit = (prob_id // 3) + 1 # 1 sec for 10, 2 for 15, 3 for 20  
    modelled_problem = modelled_problems[prob_id]
    modelled_problem.solve(time_limit=initial_solution_time_limit)
    initial_solutions_dict[prob_id] = modelled_problem.extract_solution()

Set parameter TimeLimit to value 1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 4127 rows, 680 columns and 19367 nonzeros
Model fingerprint: 0x92cb04c1
Variable types: 0 continuous, 680 integer (575 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 8e+02]
  RHS range        [1e+00, 8e+02]
Presolve removed 570 rows and 154 columns
Presolve time: 0.05s
Presolved: 3557 rows, 526 columns, 13502 nonzeros
Variable types: 0 continuous, 526 integer (426 binary)
Found heuristic solution: objective 1028.0000000

Root relaxation: objective 1.603000e+02, 347 iterations, 0.02 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  160.30000    0   27 1028.00000  160.30000  84.4% 

In [18]:
import time

In [19]:
def get_gap(exact_val, heuristic_val):
    zp = heuristic_val
    zd = exact_val

    return abs(zp - zd) / abs(zp)

In [20]:
# Diversification
time_limit = 100
rel_problem_ids = [3, 6]
nb_processes_list = [1, 2, 3, 4, 5, 6]

lns_div_solvers = dict()
for prob_id in rel_problem_ids:
    problem = problems[prob_id]
    init_sol = initial_solutions_dict[prob_id]
    
    prob_solvers = []
    for nb_process in nb_processes_list:
        plns = PLNS(problem, init_sol, time_limit=time_limit, nb_processes=nb_process)
        plns.set_destruction_degree_criterion(DestructionDegreeCriterion.CONSTANT)
        prob_solvers.append(plns)

    lns_div_solvers[prob_id] = prob_solvers


Parallel LNS initialized with 1 processes
Parallel LNS initialized with 2 processes
Parallel LNS initialized with 3 processes
Parallel LNS initialized with 4 processes
Parallel LNS initialized with 5 processes
Parallel LNS initialized with 6 processes
Parallel LNS initialized with 1 processes
Parallel LNS initialized with 2 processes
Parallel LNS initialized with 3 processes
Parallel LNS initialized with 4 processes
Parallel LNS initialized with 5 processes
Parallel LNS initialized with 6 processes


In [21]:
for prob_id in rel_problem_ids:
    for solver in lns_div_solvers[prob_id]:
        solver.search()

Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 645 iterations
Total time: 100.00 Best objective: 301 found in 572 iterations in 89.62 s

Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 625 iterations
Total time: 100.00 Best objective: 301 found in 318 iterations in 53.61 s

Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 496 iterations
Total time: 100.00 Best objective: 277 found in 136 iterations in 28.25 s

Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 406 iterations
Total time: 100.00 Best objective: 341 found in 34 iterations in 9.49 s

Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 321 iterations
Total time: 100.00 Best objective: 307 found in 121 iterations in 33.59 s

Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 306 iterations
Total time: 100.00 Best objective

In [24]:
def get_solver_stats(solver):
    objectives = [sol.objective for sol in solver.solutions_cache]
    iterations = [sol.iteration for sol in solver.solutions_cache]
    time_found = [
        "{:.2f}".format(sol.time_to_find) for sol in solver.solutions_cache
    ]

    improving_solutions = []
    best_obj = solver.solutions_cache[0].objective
    for i, sol in enumerate(solver.solutions_cache):
        if sol.objective < best_obj:
            best_obj = sol.objective
            improving_solutions.append(sol)

    improving_iterations = [sol.iteration for sol in improving_solutions]
    improving_objectives = [sol.objective for sol in improving_solutions]
    improving_times = [
        "{:.2f}".format(sol.time_to_find) for sol in improving_solutions
    ]

    stats = dict(
        objectives=objectives,
        iterations=iterations,
        times_found=time_found,
        improving_iterations=improving_iterations,
        improving_objectives=improving_objectives,
        improving_times=improving_times,
    )

    return stats

In [68]:
# Define colors for processes,more processes darker shade

proc_colors = ["#BCD2E8", "#91BAD6", "#73A5C6", "#528AAE", "#2E5984", "#1E3F66"] + [None] * 4 + ["#000000"]

fig = make_subplots(
        rows=2,
        cols=2,
        # y_title="Objective",
        # x_title="Time(s)",
        shared_yaxes=True,
        subplot_titles=("Problem 4: All Solutions", "Problem 4: Improving Solutions",
                        "Problem 7: All Solutions", "Problem 7: Improving Solutions",
        ),
        # shared_xaxes=True,
        horizontal_spacing=0.075,
        vertical_spacing=0.1,

    )

show_all = True 
# first row, problem of 15 items, id = 3
prob_0_div_solvers = lns_div_solvers[3]

for solver in prob_0_div_solvers:
    stats = get_solver_stats(solver)
    nb_process = solver.nb_processes

    # improving solutions
    fig.add_trace(go.Scatter(x=stats["improving_times"],
                    y=stats["improving_objectives"],
                    name=f"{nb_process} process(es) (impro)",
                    marker_color=proc_colors[nb_process - 1],
                    mode="lines+markers",
                    ),
        row=1,
        col=2)

    if show_all:
        # all solutions
        fig.add_trace(go.Scatter(x=stats["times_found"],
                            y=stats["objectives"],
                            name=f"{nb_process} process(es) (all)",
                            opacity=0.5,
                            marker_color=proc_colors[nb_process - 1],
                            mode="lines",
                            ),
                row=1,
                col=1)

# second row, problem of 20 items, id = 6
prob_6_div_solvers = lns_div_solvers[6]
prob_6_conv_solvers = lns_conv_solvers[6]

for solver in prob_6_div_solvers:
    stats = get_solver_stats(solver)
    nb_process = solver.nb_processes

    # improving solutions
    fig.add_trace(go.Scatter(x=stats["improving_times"],
                    y=stats["improving_objectives"],
                    name=f"{nb_process} process(es) (impro)",
                    marker_color=proc_colors[nb_process - 1],
                    mode="lines+markers",
                    ),
        row=2,
        col=2)

    if show_all:
    # all solutions
        fig.add_trace(go.Scatter(x=stats["times_found"],
                            y=stats["objectives"],
                            name=f"{nb_process} process(es) (all)",
                            opacity=0.5,
                            marker_color=proc_colors[nb_process - 1],
                            mode="lines",
                            ),
                row=2,
                col=1)

for r in [1, 2]:
    for c in [1, 2]:    
        fig.update_yaxes(title_text="Objective", row=r, col=c)
        fig.update_xaxes(title_text="Time(s)", row=r, col=c)

names = set()
fig.for_each_trace(lambda trace: trace.update(showlegend=False)
                   if (trace.name in names) else names.add(trace.name))

fig.update_layout(height=1000, width=1200, title_text="Objective Value as PLNS Progresses")

In [83]:
prob_4_solvers = lns_div_solvers[3]
prob_7_solvers = lns_div_solvers[6]

# For problem 4
nb_processes_set = {solver.nb_processes for solver in prob_4_solvers}
nb_processes_to_iteration_times = {
    nb_process: [
        sol.iteration_time
        for solver in prob_4_solvers for sol in solver.solutions_cache if solver.nb_processes == nb_process
        if sol.iteration_time > 1e-5
    ] 
    for nb_process in nb_processes_set
} 

# Box plot for iteration times per number of process for degree of destruction = 2
fig = go.Figure()
for nb_proc, times in nb_processes_to_iteration_times.items():
    y_values = [t * 1000 for t in times]
    fig.add_trace(go.Box(y=y_values, name=nb_proc, marker_color=proc_colors[nb_proc - 1], showlegend=False))
    fig.update_xaxes(title_text="Number of Processes")
    fig.update_yaxes(title_text="Iteration Time(ms)")

fig.update_layout(height=600, width=800, title_text="Box Plot for Iteration Times for Problem 4")

fig.show()

In [84]:
# For problem 7
nb_processes_set = {solver.nb_processes for solver in prob_7_solvers}
nb_processes_to_iteration_times = {
    nb_process: [
        sol.iteration_time
        for solver in prob_7_solvers for sol in solver.solutions_cache if solver.nb_processes == nb_process
        if sol.iteration_time > 1e-5
    ] 
    for nb_process in nb_processes_set
} 

# Box plot for iteration times per number of process for degree of destruction = 2
fig = go.Figure()
for nb_proc, times in nb_processes_to_iteration_times.items():
    y_values = [t * 1000 for t in times]
    fig.add_trace(go.Box(y=y_values, name=nb_proc, marker_color=proc_colors[nb_proc - 1], showlegend=False))
    fig.update_xaxes(title_text="Number of Processes")
    fig.update_yaxes(title_text="Iteration Time(ms)")

fig.update_layout(height=600, width=800, title_text="Box Plot for Iteration Times for Problem 7")

In [90]:
exact_prob_4 = 215
exact_prob_7 = 193

prob_4_gaps = dict()
for solver in prob_4_solvers:
    obj = solver.best_cached_solution.objective
    last_iteration = solver.solutions_cache[-1].iteration
    value = get_gap(solver.best_cached_solution.objective, exact_prob_4) * 100
    prob_4_gaps[solver.nb_processes] = (obj, "{:.2f}".format(value), last_iteration)
    print(f"Prob 4: nb_pro: {solver.nb_processes}, iters: {last_iteration}, obj: {obj}, gap: {'{:.2f}'.format(value)}")

print()
prob_7_gaps = dict()
for solver in prob_7_solvers:
    obj = solver.best_cached_solution.objective
    last_iteration = solver.solutions_cache[-1].iteration
    value = get_gap(solver.best_cached_solution.objective, exact_prob_7) * 100
    prob_4_gaps[solver.nb_processes] = (obj, "{:.2f}".format(value), last_iteration)
    print(f"Prob 7: nb_pro: {solver.nb_processes}, iters: {last_iteration}, obj: {obj}, gap: {'{:.2f}'.format(value)}")



Prob 4: nb_pro: 1, iters: 644, obj: 301, gap: 40.00
Prob 4: nb_pro: 2, iters: 624, obj: 301, gap: 40.00
Prob 4: nb_pro: 3, iters: 495, obj: 277, gap: 28.84
Prob 4: nb_pro: 4, iters: 405, obj: 341, gap: 58.60
Prob 4: nb_pro: 5, iters: 320, obj: 307, gap: 42.79
Prob 4: nb_pro: 6, iters: 305, obj: 305, gap: 41.86

Prob 7: nb_pro: 1, iters: 147, obj: 339, gap: 75.65
Prob 7: nb_pro: 2, iters: 136, obj: 293, gap: 51.81
Prob 7: nb_pro: 3, iters: 123, obj: 323, gap: 67.36
Prob 7: nb_pro: 4, iters: 107, obj: 266, gap: 37.82
Prob 7: nb_pro: 5, iters: 90, obj: 269, gap: 39.38
Prob 7: nb_pro: 6, iters: 83, obj: 287, gap: 48.70
